In [3]:
import nltk

# Download tokenizer data if not already available
nltk.download('punkt')

# 1. Tokenize paragraph
paragraph = "The quick brown fox jumps over the lazy dog. It's a sunny day, and everyone is happy! Let's go outside."
naive_tokens = paragraph.split()
manual_tokens = [
    "The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog", ".",
    "It", "'s", "a", "sunny", "day", ",", "and", "everyone", "is", "happy", "!",
    "Let", "'s", "go", "outside", "."
]

# 2. Compare with NLTK
tool_tokens_nltk = nltk.word_tokenize(paragraph)

def compare_tokens(manual, tool):
    return [(m, t) for m, t in zip(manual, tool) if m != t]

nltk_diff = compare_tokens(manual_tokens, tool_tokens_nltk)

# 3. MWEs
mwes = [
    ("New York", "Place name"),
    ("kick the bucket", "Idiom"),
    ("machine learning", "Technical phrase")
]

# 4. Reflection
reflection = (
    "Hardest part was punctuation and contractions. "
    "English clitics split differently by tools. "
    "Morphology in other languages adds complexity. "
    "Naive split merges punctuation with words. "
    "MWEs are tricky since tools split them but meaning requires them whole. "
    "Tokenization needs linguistic awareness."
)

# Output
print("Naive tokens:", naive_tokens)
print("\nManual tokens:", manual_tokens)
print("\nNLTK tokens:", tool_tokens_nltk)
print("\nDifferences (Manual vs NLTK):", nltk_diff)
print("\nMultiword Expressions:", mwes)
print("\nReflection:\n", reflection)


Naive tokens: ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog.', "It's", 'a', 'sunny', 'day,', 'and', 'everyone', 'is', 'happy!', "Let's", 'go', 'outside.']

Manual tokens: ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.', 'It', "'s", 'a', 'sunny', 'day', ',', 'and', 'everyone', 'is', 'happy', '!', 'Let', "'s", 'go', 'outside', '.']

NLTK tokens: ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.', 'It', "'s", 'a', 'sunny', 'day', ',', 'and', 'everyone', 'is', 'happy', '!', 'Let', "'s", 'go', 'outside', '.']

Differences (Manual vs NLTK): []

Multiword Expressions: [('New York', 'Place name'), ('kick the bucket', 'Idiom'), ('machine learning', 'Technical phrase')]

Reflection:
 Hardest part was punctuation and contractions. English clitics split differently by tools. Morphology in other languages adds complexity. Naive split merges punctuation with words. MWEs are tricky since tools split them but meaning requires

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chava\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# mini_bpe.py
from collections import Counter
import re

def get_word_tokens(corpus_words):
    return [tuple(list(w) + ['_']) for w in corpus_words]

def get_pair_counts(word_tokens):
    counts = Counter()
    for wt in word_tokens:
        for i in range(len(wt)-1):
            counts[(wt[i], wt[i+1])] += 1
    return counts

def merge_pair_in_tokens(word_tokens, pair):
    a,b = pair
    merged = a+b
    new_tokens = []
    for wt in word_tokens:
        new_w = []
        i=0
        while i < len(wt):
            if i < len(wt)-1 and wt[i]==a and wt[i+1]==b:
                new_w.append(merged)
                i += 2
            else:
                new_w.append(wt[i])
                i += 1
        new_tokens.append(tuple(new_w))
    return new_tokens

def learn_bpe(word_list, num_merges=10):
    wtoks = get_word_tokens(word_list)
    vocab = set(sorted(set("".join(word_list)) | {'_'}))
    merge_ops = []
    for i in range(num_merges):
        counts = get_pair_counts(wtoks)
        if not counts:
            break
        top_pair, freq = counts.most_common(1)[0]
        new_tok = top_pair[0] + top_pair[1]
        merge_ops.append((top_pair, freq, new_tok))
        vocab.add(new_tok)
        wtoks = merge_pair_in_tokens(wtoks, top_pair)
        print(f"Merge {i+1}: {top_pair} (freq={freq}) -> '{new_tok}' | Vocab size: {len(vocab)}")
    return merge_ops, wtoks, vocab

def apply_merges_list(word, merge_ops):
    toks = tuple(list(word) + ['_'])
    for pair,_,_ in merge_ops:
        toks = merge_pair_in_tokens([toks], pair)[0]
    return list(toks)

# Example use:
toy = "low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new".split()
merge_ops, tokens, vocab = learn_bpe(toy, num_merges=10)
print("Learned merges:", [(p,f,n) for p,f,n in merge_ops])
for w in ["new","newer","lowest","widest","newestest"]:
    print(w, "->", apply_merges_list(w, merge_ops))


Merge 1: ('e', 'r') (freq=9) -> 'er' | Vocab size: 12
Merge 2: ('er', '_') (freq=9) -> 'er_' | Vocab size: 13
Merge 3: ('n', 'e') (freq=8) -> 'ne' | Vocab size: 14
Merge 4: ('ne', 'w') (freq=8) -> 'new' | Vocab size: 15
Merge 5: ('l', 'o') (freq=7) -> 'lo' | Vocab size: 16
Merge 6: ('lo', 'w') (freq=7) -> 'low' | Vocab size: 17
Merge 7: ('new', 'er_') (freq=6) -> 'newer_' | Vocab size: 18
Merge 8: ('low', '_') (freq=5) -> 'low_' | Vocab size: 19
Merge 9: ('w', 'i') (freq=3) -> 'wi' | Vocab size: 20
Merge 10: ('wi', 'd') (freq=3) -> 'wid' | Vocab size: 21
Learned merges: [(('e', 'r'), 9, 'er'), (('er', '_'), 9, 'er_'), (('n', 'e'), 8, 'ne'), (('ne', 'w'), 8, 'new'), (('l', 'o'), 7, 'lo'), (('lo', 'w'), 7, 'low'), (('new', 'er_'), 6, 'newer_'), (('low', '_'), 5, 'low_'), (('w', 'i'), 3, 'wi'), (('wi', 'd'), 3, 'wid')]
new -> ['new', '_']
newer -> ['newer_']
lowest -> ['low', 'e', 's', 't', '_']
widest -> ['wid', 'e', 's', 't', '_']
newestest -> ['new', 'e', 's', 't', 'e', 's', 't', '_']


In [5]:
# Q3: Manual & Mini BPE Implementation

from collections import Counter
import re

# --- Helper functions ---
def get_word_tokens(corpus_words):
    return [tuple(list(w) + ['_']) for w in corpus_words]

def get_pair_counts(word_tokens):
    counts = Counter()
    for wt in word_tokens:
        for i in range(len(wt)-1):
            counts[(wt[i], wt[i+1])] += 1
    return counts

def merge_pair_in_tokens(word_tokens, pair):
    a,b = pair
    merged = a+b
    new_tokens = []
    for wt in word_tokens:
        new_w = []
        i=0
        while i < len(wt):
            if i < len(wt)-1 and wt[i]==a and wt[i+1]==b:
                new_w.append(merged)
                i += 2
            else:
                new_w.append(wt[i])
                i += 1
        new_tokens.append(tuple(new_w))
    return new_tokens

def learn_bpe(word_list, num_merges=10):
    wtoks = get_word_tokens(word_list)
    vocab = set(sorted(set("".join(word_list)) | {'_'}))
    merge_ops = []
    for i in range(num_merges):
        counts = get_pair_counts(wtoks)
        if not counts: break
        top_pair, freq = counts.most_common(1)[0]
        new_tok = top_pair[0] + top_pair[1]
        merge_ops.append((top_pair, freq, new_tok))
        vocab.add(new_tok)
        wtoks = merge_pair_in_tokens(wtoks, top_pair)
        print(f"Merge {i+1}: {top_pair} (freq={freq}) -> '{new_tok}' | Vocab size: {len(vocab)}")
    return merge_ops, wtoks, vocab

def apply_merges_list(word, merge_ops):
    toks = tuple(list(word) + ['_'])
    for pair,_,_ in merge_ops:
        toks = merge_pair_in_tokens([toks], pair)[0]
    return list(toks)


# --- 3.1 Manual BPE on toy corpus ---
toy = "low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new".split()
print("=== 3.1 Manual BPE ===")
print("Corpus:", toy)

tokens = get_word_tokens(toy)
vocab = set(sorted(set("".join(toy)) | {"_"}))
print("Initial vocabulary:", sorted(list(vocab)))

for step in range(1,4):
    counts = get_pair_counts(tokens)
    pair, freq = counts.most_common(1)[0]
    merged = pair[0]+pair[1]
    print(f"\nStep {step}: top pair {pair} freq={freq} -> '{merged}'")
    before = ["".join(t) for t in tokens][:8]
    tokens = merge_pair_in_tokens(tokens, pair)
    after = ["".join(t) for t in tokens][:8]
    vocab.add(merged)
    print("  before:", before)
    print("  after :", after)
    print("  vocab size:", len(vocab))


# --- 3.2 Mini-BPE learner ---
print("\n=== 3.2 Mini-BPE learner ===")
merge_ops, final_tokens, final_vocab = learn_bpe(toy, num_merges=10)

words_to_segment = ["new", "newer", "lowest", "widest", "newestest"]
print("\nSegmentations:")
for w in words_to_segment:
    print(f"{w} -> {apply_merges_list(w, merge_ops)}")

print("\nReflection (3.2):")
print("- Subwords solve OOV by breaking unseen words into known parts.")
print("- Example: 'er_' corresponds to a real morpheme (-er suffix).")
print("- This makes BPE useful for rare or new forms.")


# --- 3.3 Train BPE on paragraph ---
print("\n=== 3.3 Paragraph BPE (30 merges) ===")
paragraph = (
    "Artificial intelligence is changing how we work and learn. "
    "Researchers build models that can generate text, translate languages, and assist creativity. "
    "Small datasets still teach useful patterns, especially when using subword methods. "
    "Subword tokenization helps with rare words and morphological variations. "
    "Practitioners tune merge operations to balance vocabulary size and coverage."
)
words = re.findall(r"\b\w+\b", paragraph.lower())
print("Training words:", words[:20], "...")

merge_ops_para, final_tokens_para, final_vocab_para = learn_bpe(words, num_merges=30)

print("\nTop 5 merges:")
for i,(pair,freq,newtok) in enumerate(merge_ops_para[:5],1):
    print(f"{i}. {pair} -> {newtok}")

sorted_vocab_by_len = sorted(final_vocab_para, key=lambda x: (-len(x), x))
print("\nFive longest subword tokens:")
for tok in sorted_vocab_by_len[:5]:
    print(tok)

words_sample = ["intelligence", "researchers", "subword", "variations", "practitioners"]
print("\nSegmentations for sample words:")
for w in words_sample:
    print(f"{w} -> {apply_merges_list(w, merge_ops_para)}")

print("\nReflection (3.3):")
print("- Learned subwords include stems, suffixes (e.g., 'ation'), and frequent whole words.")
print("- Pros: handles rare/derived words, reduces vocab size.")
print("- Cons: sometimes splits inside morphemes; not always linguistically clean.")
print("- Tokenization difficulty depends on morphology and MWEs in the language.")


=== 3.1 Manual BPE ===
Corpus: ['low', 'low', 'low', 'low', 'low', 'lowest', 'lowest', 'newer', 'newer', 'newer', 'newer', 'newer', 'newer', 'wider', 'wider', 'wider', 'new', 'new']
Initial vocabulary: ['_', 'd', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w']

Step 1: top pair ('e', 'r') freq=9 -> 'er'
  before: ['low_', 'low_', 'low_', 'low_', 'low_', 'lowest_', 'lowest_', 'newer_']
  after : ['low_', 'low_', 'low_', 'low_', 'low_', 'lowest_', 'lowest_', 'newer_']
  vocab size: 12

Step 2: top pair ('er', '_') freq=9 -> 'er_'
  before: ['low_', 'low_', 'low_', 'low_', 'low_', 'lowest_', 'lowest_', 'newer_']
  after : ['low_', 'low_', 'low_', 'low_', 'low_', 'lowest_', 'lowest_', 'newer_']
  vocab size: 13

Step 3: top pair ('n', 'e') freq=8 -> 'ne'
  before: ['low_', 'low_', 'low_', 'low_', 'low_', 'lowest_', 'lowest_', 'newer_']
  after : ['low_', 'low_', 'low_', 'low_', 'low_', 'lowest_', 'lowest_', 'newer_']
  vocab size: 14

=== 3.2 Mini-BPE learner ===
Merge 1: ('e', 'r') (freq=9) 

In [7]:
# Q4. Edit Distance between "Sunday" -> "Saturday"
# Covers: Model A (sub=1, ins=1, del=1) and Model B (sub=2, ins=1, del=1)

def edit_distance_and_ops(s, t, sub_cost=1, ins_cost=1, del_cost=1):
    m, n = len(s), len(t)
    dp = [[0]*(n+1) for _ in range(m+1)]
    op = [[None]*(n+1) for _ in range(m+1)]

    # Initialize DP base cases
    for i in range(1, m+1):
        dp[i][0] = i * del_cost
        op[i][0] = ('del', s[i-1])
    for j in range(1, n+1):
        dp[0][j] = j * ins_cost
        op[0][j] = ('ins', t[j-1])

    # Fill DP table
    for i in range(1, m+1):
        for j in range(1, n+1):
            cost_sub = dp[i-1][j-1] + (0 if s[i-1]==t[j-1] else sub_cost)
            cost_del = dp[i-1][j] + del_cost
            cost_ins = dp[i][j-1] + ins_cost
            best = min(cost_sub, cost_del, cost_ins)
            dp[i][j] = best
            if best == cost_sub:
                op[i][j] = ('match' if s[i-1]==t[j-1] else 'sub', (s[i-1], t[j-1]))
            elif best == cost_del:
                op[i][j] = ('del', s[i-1])
            else:
                op[i][j] = ('ins', t[j-1])

    # Backtrace sequence
    i, j = m, n
    seq = []
    while i > 0 or j > 0:
        a = op[i][j]
        if a is None:
            break
        if a[0] in ('match','sub'):
            seq.append(a)
            i -= 1; j -= 1
        elif a[0]=='del':
            seq.append(a)
            i -= 1
        else:
            seq.append(a)
            j -= 1
    seq.reverse()
    return dp, seq

def print_dp_table(dp, s, t):
    # Pretty print DP matrix
    print("    " + "  ".join("#"+c for c in "#"+t))
    for i,row in enumerate(dp):
        label = "#" if i==0 else s[i-1]
        print(label, " ".join(f"{cell:2d}" for cell in row))

# --- Run for both models ---
s, t = "Sunday", "Saturday"

print("=== Model A: sub=1, ins=1, del=1 ===")
dpA, seqA = edit_distance_and_ops(s, t, sub_cost=1, ins_cost=1, del_cost=1)
print("Edit distance =", dpA[len(s)][len(t)])
print("\nDP Table:")
print_dp_table(dpA, s, t)
print("\nOne valid edit sequence:")
for step in seqA:
    print(" ", step)

print("\n=== Model B: sub=2, ins=1, del=1 ===")
dpB, seqB = edit_distance_and_ops(s, t, sub_cost=2, ins_cost=1, del_cost=1)
print("Edit distance =", dpB[len(s)][len(t)])
print("\nDP Table:")
print_dp_table(dpB, s, t)
print("\nOne valid edit sequence:")
for step in seqB:
    print(" ", step)

# --- Reflection ---
print("\n=== Reflection ===")
print("- Model A distance = 3, Model B distance = 4.")
print("- In Model A substitution was cheap, so we used it.")
print("- In Model B substitution was expensive, so insert/delete became equally good.")
print("- Insertions were necessary for 'a' and 't'.")
print("- Applications differ: spell check prefers Model A (subs common),")
print("  DNA alignment prefers Model B (insertions/deletions more realistic).")


=== Model A: sub=1, ins=1, del=1 ===
Edit distance = 3

DP Table:
    ##  #S  #a  #t  #u  #r  #d  #a  #y
#  0  1  2  3  4  5  6  7  8
S  1  0  1  2  3  4  5  6  7
u  2  1  1  2  2  3  4  5  6
n  3  2  2  2  3  3  4  5  6
d  4  3  3  3  3  4  3  4  5
a  5  4  3  4  4  4  4  3  4
y  6  5  4  4  5  5  5  4  3

One valid edit sequence:
  ('match', ('S', 'S'))
  ('ins', 'a')
  ('ins', 't')
  ('match', ('u', 'u'))
  ('sub', ('n', 'r'))
  ('match', ('d', 'd'))
  ('match', ('a', 'a'))
  ('match', ('y', 'y'))

=== Model B: sub=2, ins=1, del=1 ===
Edit distance = 4

DP Table:
    ##  #S  #a  #t  #u  #r  #d  #a  #y
#  0  1  2  3  4  5  6  7  8
S  1  0  1  2  3  4  5  6  7
u  2  1  2  3  2  3  4  5  6
n  3  2  3  4  3  4  5  6  7
d  4  3  4  5  4  5  4  5  6
a  5  4  3  4  5  6  5  4  5
y  6  5  4  5  6  7  6  5  4

One valid edit sequence:
  ('match', ('S', 'S'))
  ('ins', 'a')
  ('ins', 't')
  ('match', ('u', 'u'))
  ('sub', ('n', 'r'))
  ('match', ('d', 'd'))
  ('match', ('a', 'a'))
  ('match',